# Day 03 — Tool routing (structured outputs)

Today I practice **tool routing**: asking the model to choose a tool and return structured JSON.


## Goals for this notebook
- Define a small set of tools (summarize, plan, estimate).
- Ask the model to pick the best tool for a request.
- Parse the JSON and execute the chosen tool locally.

The idea is to keep the prompt tiny but make the steps explicit.


In [ ]:
import json
from pathlib import Path

from openai import OpenAI

client = OpenAI()
PROMPTS_DIR = Path("prompts")

def load_prompt(name: str) -> str:
    return (PROMPTS_DIR / name).read_text()

def render(template: str, **kwargs: str) -> str:
    text = template
    for key, value in kwargs.items():
        text = text.replace(f"{{{key}}}", value)
    return text

def call_openai(prompt: str, model: str = "gpt-4o-mini") -> str:
    response = client.responses.create(
        model=model,
        input=prompt,
    )
    return response.output_text


## Define local tools
Each tool is a plain Python function. The model will pick one and
return a JSON payload describing which tool to run.


In [ ]:
def summarize_text(text: str) -> str:
    return "Summary: " + text.split(".")[0] + "."

def estimate_budget(items: list[str]) -> str:
    return f"Estimated budget for {len(items)} items: $50"

def create_study_plan(topic: str) -> str:
    return f"3-step study plan for {topic}: 1) Basics 2) Practice 3) Review"


## Ask the model to route the request
The routing prompt lists available tools and demands JSON output.
We can then parse the JSON and call the tool function locally.


In [ ]:
request = "Create a study plan for learning linear regression in two days."

router_prompt = render(
    load_prompt("router.txt"),
    request=request,
)

tool_call = call_openai(router_prompt)
tool_call


## Parse and execute
Once we have JSON, we select the tool and run it.


In [ ]:
payload = json.loads(tool_call)
tool_name = payload["tool"]
args = payload["arguments"]

tool_map = {
    "summarize_text": summarize_text,
    "estimate_budget": estimate_budget,
    "create_study_plan": create_study_plan,
}

result = tool_map[tool_name](**args)
result
